In [ ]:
import requests
from pathlib import Path
import logging
import re
from collections import defaultdict

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Output directory
bronze_path = Path("data\\bronze\\ncdb")
bronze_path.mkdir(parents=True, exist_ok=True)

# Script for downloarding the NCDB dataset

In [3]:
def download_ncdb_datasets():
    api_url = "https://open.canada.ca/data/api/3/action/package_show?id=1eb9eba7-71d1-4b30-9fb1-30cbdab7e63a"
    
    print("1. Fetching metadata from Open Canada API...")
    try:
        response = requests.get(api_url)
        data = response.json()
        resources = data['result']['resources']
    except Exception as e:
        print(f"❌ API Error: {e}")
        return

    # 2. Group Resources by Year
    # We want to collect ALL possible URLs for "2021", "2020", etc.
    year_pattern = re.compile(r'(19|20)\d{2}')
    resources_by_year = defaultdict(list)

    for r in resources:
        name = r.get('name', '')
        url = r.get('url', '')
        fmt = r.get('format', '').upper()

        # Skip dictionaries and non-CSVs
        if 'CSV' not in fmt or 'dictionary' in name.lower() or 'dictionnaire' in name.lower():
            continue

        # Find year in name
        match = year_pattern.search(name)
        if match:
            year = match.group(0)
            # Special case for the "99_TO_2001" file -> This covers 1999, 2000, 2001
            # We'll label it "1999_2001" to be safe, or just attach it to the first year found.
            # For simplicity, if the name says "99_TO_2001", let's save it as that.
            if "99_TO" in name:
                year = "1999_2001"
            
            if url and url.startswith("http"):
                resources_by_year[year].append(url)

    print(f"2. Found {len(resources_by_year)} unique year-groups to process.")

    # 3. Download One Good File Per Year
    for year, urls in resources_by_year.items():
        output_file = bronze_path / f"ncdb_{year}.csv"
        
        if output_file.exists():
            print(f"    {year}: File already exists. Skipping.")
            continue
        
        # Try URLs until one works
        success = False
        for i, url in enumerate(urls):
            print(f"    {year}: Attempting download (Source {i+1})...")
            try:
                r = requests.get(url, stream=True, timeout=30)
                r.raise_for_status()
                
                with open(output_file, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                print(f"    {year}: Success!")
                success = True
                break # Stop trying other URLs for this year
            except Exception as e:
                print(f"       Source {i+1} failed: {e}")
        
        if not success:
            print(f"    {year}: All download links failed.")

    print("\n Download process finished.")

In [4]:
download_ncdb_datasets()

1. Fetching metadata from Open Canada API...
2. Found 4 unique year-groups to process.
    2019: Attempting download (Source 1)...
    2019: Success!
    1999_2001: Attempting download (Source 1)...
    1999_2001: Success!
    2020: Attempting download (Source 1)...
    2020: Success!
    2021: Attempting download (Source 1)...
    2021: Success!

 Download process finished.
